# Shor's order-finding algorithm
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Shor's order-finding algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), modified as in [[E24]](https://doi.org/10.1145/3655026), and with the classical post-processing from [[E24]](https://doi.org/10.1145/3655026).
It furthermore exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Seifert's variation [[Seifert01]](https://doi.org/10.1007/3-540-45353-9_24) of Shor's order-finding algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), as described in [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A), and with the classical post-processing from [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A) with supporting functions from [[E24]](https://doi.org/10.1145/3655026).

To start off, let us pick an $m$-bit order $r$ uniformly at random from the set of all $m$-bit integers.
To this end, we use the [<code>sample_l_bit_integer(l)</code>](../docs/math/random/sample_l_bit_integer.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).
We then use the [SimulatedCyclicGroupElement](../docs/math/groups/SimulatedCyclicGroupElement.md) class provided by [Quaspy](https://github.com/ekera/quaspy) to define a group element $g$ of order $r$.

In [1]:
!pip3 install -q quaspy # Make sure that quaspy is installed.

In [2]:
from quaspy.math.random import sample_l_bit_integer;
from quaspy.math.groups import SimulatedCyclicGroupElement;

m = 1024;
r = sample_l_bit_integer(m);

print("Sampled r =", r);

g = SimulatedCyclicGroupElement(r);

Sampled r = 131594841169087245885790758246089422061948879052180031265533163939184466531826374047249324097036644162632131124321529677437673729496087845655613048388683302644957056342791826987364296180668671248217191072822797814278429027392341280616507749774237206697340161365861616561794070658913963075325317958216733943409


## 1. Solving for the order $r$ in a single run

### 1.1. Sampling a frequency $j$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) for simulating Shor's order-finding algorithm (and Seifert's variation thereof) exactly for a given order $r$.
For further details, see [[E24t]](https://kth.diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sects. 5.3.5 and 5.4.3).

Below, we use said function to simulate running the algorithm for $r$ with a control register of $m + \ell$ qubits, where $m$ is an upper bound on the bit length of $r$, and $\ell$ is a positive integer.

More specifically, for $g$ a generator of order $r < 2^m$, said function simulates inducing the state

$$\frac{1}{2^{m + \ell}}
\sum_{a, \, j \, = \, 0}^{2^{m+\ell} - 1}
\mathrm{exp}
\left(
  \frac{2 \pi \mathrm{i}}{2^{m + \ell}} aj
\right)
|\, j, g^a \,\rangle$$

and reading out the first control register.
This yields a frequency $j$ sampled from the probability distribution induced by the quantum part of the order-finding algorithm:

In [3]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m.

j = sample_j_given_r(r, m, l, timeout = 30);

print("Sampled j =", j);

Sampled j = 26567545896586009355811451821647630833833997889255733411282712512882270830975851274472504392189351467119866868893207042485733105103764689564970275268126068059033674127471172365534019396844787561692094772041061218361704526694397709763170631452988481439691873096440089779892277146507738391333291754186666640467088756147216294574590123367637008143494288154746059831338805624228576471593217894857763274776948856693537613318354139685623655443330082246258944379829154465918578587663116189024945971515858215561848169301618082780084785198594385798325898934032729718541226593962846700498794932189102725074389397063556853628472


### 1.2. Solving the frequency $j$ and $g$ for $r$

We may then proceed to solve $j$ and $g$ for the order $r$ by calling the [<code>solve_j_for_r(j, m, l, g, ..)</code>](../docs/orderfinding/general/postprocessing/ekera/solve_j_for_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy).

By default this function uses the lattice-based post-processing from [[E24]](https://doi.org/10.1145/3655026) without enumerating the lattice, in combination with searching offsets in $j$, and reconstructing the missing part of the order when smooth, and so forth.
For the full details and a lower bound on the success probability, see [[E24]](https://doi.org/10.1145/3655026).
Note that if we accept to enumerate the lattice, we can decrease $\ell$, again see [[E24]](https://doi.org/10.1145/3655026) for the full details.

This yields the order $r$: 

In [4]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r;

result = solve_j_for_r(j, m, l, g);

print("Solving for r yielded:", result);

if result == r:
  print("\n[ OK ] The order r was successfully recovered.");
else:
  print("\n[FAIL] The order r was not recovered.");

Solving for r yielded: 131594841169087245885790758246089422061948879052180031265533163939184466531826374047249324097036644162632131124321529677437673729496087845655613048388683302644957056342791826987364296180668671248217191072822797814278429027392341280616507749774237206697340161365861616561794070658913963075325317958216733943409

[ OK ] The order r was successfully recovered.


## 2. Making tradeoffs and solving for $r$ in multiple runs
Let us now consider Seifert's variation [[Seifert01]](https://doi.org/10.1007/3-540-45353-9_24) of Shor's order-finding algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), as described in [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A), where the idea is to make tradeoffs by picking $\ell \approx m/s$ for $s$ some tradeoff factor instead of picking $\ell \sim m$.

As explained in [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A), each run of the quantum part of the algorithm yields at least $\sim \ell$ bits of information on the order $r$.
Hence, we expect to need approximately $s$ runs to solve for $r$ efficiently and with high probability of success in the classical post-processing.

According to the estimates in [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see Tabs. A1–A2) which were computed with the [Qunundrum](https://github.com/ekera/qunundrum) suite of MPI programs, when $m = 1024$, $s = 8$ and $\ell = \lceil m / s \rceil$, we need to make no more than $n = 9$ runs to solve efficiently in the classical post-processing with $\ge 99\%$ success probability without enumerating the lattice.
This when using the lattice-based post-processing from [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A) with supporting functions from [[E24]](https://doi.org/10.1145/3655026).
In the below example, we use this specific parameterization.

### 2.1. Sampling $n$ frequencies $(j_1, \, \ldots, \, j_n)$
To start off, we proceed in analogy with Sect. 1.1 above to sample $n = 9$ frequencies $(j_1, \, \ldots, \, j_n)$ from the distribution induced by the quantum part of the algorithm.
To this end, we use the [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy).

In [5]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

from math import ceil;

s = 8;
l = ceil(m / s);
n = 9;

j_list = [sample_j_given_r(r, m, l, timeout = 30) for _ in range(n)];

print("Sampled j =", j_list);

Sampled j = [14876088871139495494786621584206336889218613329728092432077113431101975651172211170547817022459559362108061632633815583098024940955645704213746774664394208742813854328250565895054876706068828765141657764334441081762352950933497487382658751133804375748206003037895051197236996038170599199893286868529158270183960391877479521898284244400207978461657, 39059570833644969979972426180565297276745799493021272147930108343014679465326001402106764252263649680939702033108230825125921033876723906206494621189827967834505824238742238595549593344306877351746660003952723873366801129265148410198053519996305858161052467964411159413219300743357860223571069054851459787258530650465294444211297612820241273225965, 1657278570263856400091522684868141130912852010062811260895886919026022485181095554803469818278270599732187242017646694676771095150355064731853349641639635997159882120285290591955309399350061777366934866091545563933590551147651448599511315946519609611080520376749186094314013111974247634507

### 2.2. Solving the $n$ frequencies $(j_1, \, \ldots, \, j_n)$ and $g$ for $r$
We may then proceed to solve the $n$ frequencies $(j_1, \, \ldots, \, j_n)$ and $g$ for the order $r$.

To this end, we use the [<code>solve_multiple_j_for_r(j_list, m, l, g, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_multiple_j_k_for_d.md) function provided by [Quaspy](https://github.com/ekera/quaspy). It jointly solves  $(j_1, \, \ldots, \, j_n)$ for $r$ by using the lattice-based post-processing from [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.4) and [[E21]](https://doi.org/10.1515/jmc-2020-0006) (see App. A) with supporting functions from [[E24]](https://doi.org/10.1145/3655026).

In [6]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_multiple_j_for_r;

recovered_r = solve_multiple_j_for_r(j_list,
                                     m,
                                     l,
                                     g,
                                     enumerate = False,
                                     timeout = 30);

if (recovered_r == r):
  print("Recovered r =", r);

  print("\n[ OK ] Successfully recovered r.");
else:
  print("[FAIL] Failed to recover r.");

Recovered r = 131594841169087245885790758246089422061948879052180031265533163939184466531826374047249324097036644162632131124321529677437673729496087845655613048388683302644957056342791826987364296180668671248217191072822797814278429027392341280616507749774237206697340161365861616561794070658913963075325317958216733943409

[ OK ] Successfully recovered r.
